In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
import scipy.io
from keras import backend as K
from keras.layers import Activation

In [2]:
# define the filename to save the weights of the model as a .mat
filename = 'models/homoclinic_1'

# define the subset of the (x_1,x_2)-space to approximate dynamics
x1_lower_limit = 3.5
x1_upper_limit = 6.5
x2_lower_limit = 3.5
x2_upper_limit = 6.5
step_size = 0.01;

# define the interval of the parameter species
param_lower_limit = 1.9
param_upper_limit = 2.1
step_size_par = 0.1;

# hardcode the gamma, beta, tau and  constants in the chemical perceptron (later examples automatically train this parameter)
# Note -  tau corresponds to the scaling of  -tau*y^2 (see Appendix B - (32) & (33))
gamma = 1
beta_1 = 1
tau = 1;

# define the number of chemical perceptrons species in a single hidden layer
N = 10;

# bifurication center and critical point
a = 5;
r_critical = 2

# define the number of epochs to use in training the model
number_of_epochs = 1

In [3]:
# define the custom activation function for the training with particular values of gamma and tau 
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma*tau))/tau

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})

# define training for the executive species and parameter species
x1_train = np.arange(x1_lower_limit, x1_upper_limit, step_size, dtype="float32")
x2_train = np.arange(x2_lower_limit, x2_upper_limit, step_size, dtype="float32")
x3_train = np.arange(param_lower_limit, param_upper_limit, step_size_par, dtype="float32")

x1v, x2v, x3v = np.meshgrid(x1_train, x2_train, x3_train, indexing='ij')

nx1 = len(x1_train)
nx2 = len(x2_train)
nx3 = len(x3_train)

# compute the target ODE i.e. shifted from this paper https://doi.org/10.1007/s11071-025-11622-1
g_1 = (x3v-r_critical-(4/5))*(x1v-a)+(x2v-a)-(6/5)*(x1v-a)*(x2v-a)+(3/2)*np.power((x2v-a),2)
g_2 = (x1v-a) -(4/5)*(x2v-a) - (4/5)*np.power((x2v-a),2)

# according to Algorithm 1 in https://doi.org/10.48550/arXiv.2406.03456
y1_train = (g_1 - beta_1)/x1v
y2_train = (g_2 - beta_1)/x2v

# process the array shapes 
x_train = np.append(np.append(x1v.reshape(-1,1), x2v.reshape(-1,1),axis=1), x3v.reshape(-1,1),axis=1)
y_train = np.append(y1_train.reshape(-1,1), y2_train.reshape(-1,1),axis=1)

In [4]:
# define the quasi-static approxiamtion according to Algorithm 1 in https://doi.org/10.48550/arXiv.2406.03456
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(3,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(2, activation=None, use_bias=False),
])

# compile model with an optimizer and mse loss function
model.compile(optimizer='adamax',
              loss='mse',
              metrics=['mse'])

In [5]:
# train the qusai-static neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1)

7594/7594 [==============================] - 8s 1ms/step - loss: 0.1631 - mse: 0.1631 - val_loss: 0.0485 - val_mse: 0.0485


In [6]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
  (Flatten)                  (None, 3)                 0         
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
 dense_1 (Dense)             (None, 2)                 20        
                                                                 
Total params: 60 (240.00 Byte)
Trainable params: 60 (240.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 
                                   'first_layer_biases':first_layer_biases, 
                                   'output_layer_weights':output_layer_weights,
                                   'gamma': gamma,
                                   'beta': beta_1,
                                   'alpha': tau #abuse of notation - still refering to tau  
                                  })